In [3]:
pip install -U accelerate


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import faiss
import logging
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np
import torch
from transformers import pipeline


c:\Users\HP\10 Acadamy PRojects\New folder (6)\Complaint-Analysis-RAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load Chunked Data and FAISS Index

In [3]:
# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Load chunked data
data_path = r'C:\Users\HP\10 Acadamy PRojects\New folder (6)\Complaint-Analysis-RAG\data\chunked_complaints.csv'
try:
    df_chunks = pd.read_csv(data_path)
    logging.info(f"Loaded chunked dataset with shape: {df_chunks.shape}")
    print("Columns:", df_chunks.columns.tolist())
    print("\nSample Chunks (first 5):")
    print(df_chunks[['complaint_id', 'product', 'chunk_idx', 'chunk_text']].head())
except Exception as e:
    logging.error(f"Failed to load chunked dataset: {e}")
    raise

# Load FAISS index
index_path = r'C:\Users\HP\10 Acadamy PRojects\New folder (6)\Complaint-Analysis-RAG\data\complaint_index.faiss'
try:
    index = faiss.read_index(index_path)
    logging.info(f"FAISS index size: {index.ntotal}")
except Exception as e:
    logging.error(f"Failed to load FAISS index: {e}")
    raise

2025-07-06 10:56:57,850 - INFO - Loaded chunked dataset with shape: (763160, 5)


Columns: ['complaint_id', 'product', 'chunk_idx', 'chunk_text', 'chunk_length']

Sample Chunks (first 5):
   complaint_id          product  chunk_idx  \
0      14069121      Credit Card          0   
1      14061897  Savings Account          0   
2      14047085      Credit Card          0   
3      14040217      Credit Card          0   
4      14040217      Credit Card          1   

                                          chunk_text  
0  a card was opened under my name by a fraudster...  
1  i made the mistake of using my wellsfargo debi...  
2  i have a secured credit card with citibank whi...  
3  i have a citi rewards cards the credit balance...  
4  prior to the notification about reaching my li...  


2025-07-06 10:57:00,454 - INFO - FAISS index size: 763160


# Implement the Retriever

In [5]:

# Load embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

def retrieve_chunks(query, index, df_chunks, model, top_k=5):
    try:
        # Encode query
        query_embedding = model.encode([query], show_progress_bar=False)
        # Search FAISS index
        distances, indices = index.search(np.array(query_embedding, dtype=np.float32), top_k)
        # Get corresponding chunks
        retrieved_chunks = df_chunks.iloc[indices[0]][['complaint_id', 'product', 'chunk_idx', 'chunk_text']].to_dict('records')
        return retrieved_chunks, distances[0]
    except Exception as e:
        logging.error(f"Error retrieving chunks for query '{query}': {e}")
        return [], []

# Test retriever
sample_queries = [
    "Why are people unhappy with BNPL?",
    "What are common issues with Credit Card fraud?",
    "Why do Savings Account complaints happen?"
]
for query in sample_queries:
    chunks, distances = retrieve_chunks(query, index, df_chunks, model)
    print(f"\nQuery: {query}")
    print("Retrieved Chunks:")
    for i, chunk in enumerate(chunks):
        print(f"Chunk {i+1} (Product: {chunk['product']}, Distance: {distances[i]:.4f}):")
        print(chunk['chunk_text'])

2025-07-06 10:58:09,664 - INFO - Use pytorch device_name: cpu
2025-07-06 10:58:09,665 - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2



Query: Why are people unhappy with BNPL?
Retrieved Chunks:
Chunk 1 (Product: Buy Now, Pay Later (BNPL), Distance: 0.9367):
practices of bnpl companies reporting only negative data creates an incomplete and potentially damaging picture of a consumer s creditworthiness it is my understanding that the cfpb has been looking into the bnpl sector and the unfair practices that are being used difficulty accessing assistance during financial hardship furthermore affirm does not provide easily accessible avenues for customers to seek assistance during periods of financial hardship navigating their customer service channels to request payment arrangements or other forms of support is unnecessarily difficult and frustrating this lack of transparency and accessibility exacerbates the negative impact of late payments particularly during unforeseen financial challenges i have attempted to contact them and have received no assistance
Chunk 2 (Product: Credit Card, Distance: 1.0844):
to deceive consum

In [1]:
!pip install -U bitsandbytes


# Set Up the LLM

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_name = "HuggingFaceH4/zephyr-7b-beta"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
    load_in_4bit=True,
    llm_int8_enable_fp32_cpu_offload=True,
)

llm = pipeline("text-generation", model=model, tokenizer=tokenizer)


# Example inputs
system_prompt = "You are a helpful financial assistant."
user_question = "What are the main risks in microloans?"
retrieved_chunks = [
    "Microloans often face high default rates due to limited borrower credit history.",
    "Operational costs can be disproportionately high relative to loan size."
]

# Combine them
full_prompt = f"<|system|>\n{system_prompt}\n<|user|>\n{user_question}\n<|retrieved|>\n" + "\n".join(retrieved_chunks)

# Generate
response = llm(full_prompt, max_new_tokens=200, do_sample=True, temperature=0.7)

# Extract & print output
print(response[0]["generated_text"])


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]


RuntimeError: quant_type must be nf4 on CPU, got fp4